# Eat Safe, Love

## Notebook Set Up

In [3]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [4]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [9]:
# review the collections in our database
collections = db.list_collection_names()
print("Collections in the database:")
for collection in collections:
    print(collection)

Collections in the database:
establishments
establisment


In [10]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [22]:
from pymongo import MongoClient
import json

# Connection details
host = 'localhost'
port = 27017
database_name = 'uk_food'
collection_name = 'establisment'
json_file_path = "C:/Users/DeMiko Birden/Downloads/Starter_Code (9)/Starter_Code/Resources/establishments.json"  

# Connect to MongoDB
client = MongoClient(host, port)
db = client[database_name]
collection = db[collection_name]

# Read JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Insert data into MongoDB collection
collection.insert_many(data)

In [35]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)
from pymongo import MongoClient

# Database
database_name = "uk_food"

# Construct the MongoDB URI
mongo_uri = "mongodb://localhost:27017/"

# Create a MongoClient
client = MongoClient(mongo_uri)

# Access database
database = client[database_name]

In [36]:
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

# Connect to your MongoDB instance
client = MongoClient(mongo_uri)
database = client["uk_food"]

# Assuming you want to check a specific collection
collection_name = "establishment" 

# Access the actual collection object using the correct name
actual_collection = database.get_collection(collection_name)

# 1. Query to find establishments with a hygiene score of 20
query_1 = {"scores.Hygiene": 20}

# Perform the query and get the count
count_20 = actual_collection.count_documents(query_1)

# Print the count using count_documents
print(f"Number of establishments with a hygiene score equal to 20: {count_20}")

# Display the first document in the results using pprint
document_20 = actual_collection.find_one(query_1)
print("First document with a hygiene score equal to 20:")
pprint(document_20)

# Convert the results to a Pandas DataFrame
df_20 = pd.DataFrame(list(actual_collection.find(query_1).limit(10)))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df_20.head(10))


Number of establishments with a hygiene score equal to 20: 0
First document with a hygiene score equal to 20:
None
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


In [37]:
query_1 = {"scores.Hygiene": 20}
results_1 = list(actual_collection.find(query_1).limit(10))
print(results_1)


[]


In [25]:
# Find the establishments with a hygiene score of 20
from pymongo.collection import CollectionChangeStream


query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count_20 = count_20 = collection.count_documents(query)
print(f"Number of establishments with a hygiene score equal to 20: {count_20}")



Number of establishments with a hygiene score equal to 20: 164


In [31]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame( columns=["_id", "count"])
# Display the number of rows in the DataFrame
num_rows = len(df)
print(f"Number of rows in the DataFrame: {num_rows}")
# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: [_id, count]
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [38]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = query = {"LocalAuthority": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
result_count = collection.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {result_count}")
# Display the first document in the results using pprint
result_document = collection.find_one(query)
pprint(result_document)

Number of establishments in London with RatingValue >= 4: 0
None


In [39]:
# Convert the result to a Pandas DataFrame
query = {"LocalAuthority": "London", "RatingValue": {"$gte": 4}}
# Display the number of rows in the DataFrame
result_count = collection.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {result_count}")
# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of establishments in London with RatingValue >= 4: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: [_id, count]
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [40]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query =  {
    "Latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "Longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}


sort = [("HygieneScore", 1)]  # Sort in ascending order based on hygiene score


# Print the results
print("Search results sorted by hygiene score:")
pprint(df)

Search results sorted by hygiene score:
Empty DataFrame
Columns: [_id, count]
Index: []


In [42]:
# Convert result to Pandas DataFrame
result_list = list(collection.find(query).sort(sort))

df = pd.DataFrame(result_list)


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [43]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results

hygiene_score = 0

pipeline = [
    {"$match": {"HygieneScore": hygiene_score}},
    {"$group": {"_id": "$LocalAuthority", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

result = list(collection.aggregate(pipeline))

num_documents = len(result)
print(f"Number of documents in the result: {num_documents}")

print("First 10 results:")
pprint(result[:10])


Number of documents in the result: 0
First 10 results:
[]


In [44]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result)

# Display the number of rows in the DataFrame
num_rows = len(df)
print(f"Number of rows in the DataFrame: {num_rows}")
# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
pprint(df.head(10))


Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []
